# What is a classifier?

A classifier is a machine learning model that is used to discriminate different objects based on certain features.

# Principle of Naive Bayes Classifier:

A Naive Bayes classifier is a probabilistic machine learning model that’s used for classification task. The crux of the classifier is based on the Bayes theorem.

Naive Bayes methods are a set of supervised learning algorithms based on applying Bayes` theorem with the "naive" assumption of conditional independence between every pair of features given the value of the class variable. Bayes` theorem states the following relationship, given class variable $y$ and dependent feature
vector : $x_1$ through $x_n$

$$P(y \mid x_1, \dots, x_n) = \frac{P(y) P(x_1, \dots, x_n \mid y)}{P(x_1, \dots, x_n)}$$

Using the naive conditional independence assumption that

$$P(x_i | y, x_1, \dots, x_{i-1}, x_{i+1}, \dots, x_n) = P(x_i | y)$$

for all $i$, this relationship is simplified to

$$P(y \mid x_1, \dots, x_n) = \frac{P(y) \prod_{i=1}^{n} P(x_i \mid y)}{P(x_1, \dots, x_n)}$$

Since $P(x_1, \dots, x_n)$ is constant given the input,
we can use the following classification rule:

$$ P(y \mid x_1, \dots, x_n) \propto P(y) \prod_{i=1}^{n} P(x_i \mid y)

 \Downarrow

 \hat{y} = \arg\max_y P(y) \prod_{i=1}^{n} P(x_i \mid y)$$

and we can use Maximum A Posteriori (MAP) estimation to estimate $P(y)$ and $P(x_i \mid y)$; the former is then the relative frequency of class $y$ in the training set.

The different naive Bayes classifiers differ mainly by the assumptions they make regarding the distribution of $P(x_i \mid y)$.

In spite of their apparently over-simplified assumptions, naive Bayes classifiers have worked quite well in many real-world situations, famously
document classification and spam filtering. They require a small amount of training data to estimate the necessary parameters. (For theoretical
reasons why naive Bayes works well, and on which types of data it does, see the references below.)

Naive Bayes learners and classifiers can be extremely fast compared to more sophisticated methods. The decoupling of the class conditional feature distributions means that each
distribution can be independently estimated as a one dimensional distribution. This in turn helps to alleviate problems stemming from the curse of dimensionality.

On the flip side, although naive Bayes is known as a decent classifier, it is known to be a bad estimator, so the probability outputs from $$predict_proba$$ are not to be taken too seriously.

References:

 * H. Zhang (2004). $The optimality of Naive Bayes. <https://www.cs.unb.ca/~hzhang/publications/FLAIRS04ZhangH.pdf>$_ Proc. FLAIRS.


## Gaussian Naive Bayes

$GaussianNB$ implements the Gaussian Naive Bayes algorithm for classification. The likelihood of the features is assumed to be Gaussian:

$$P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma^2_y}} \exp\left(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y}\right)$$

The parameters $\sigma_y$ and $\mu_y$ are estimated using maximum likelihood.

 >>> from sklearn.datasets import load_iris
 >>> from sklearn.model_selection import train_test_split
 >>> from sklearn.naive_bayes import GaussianNB
 >>> X, y = load_iris(return_X_y=True)
 >>> X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
 >>> gnb = GaussianNB()
 >>> y_pred = gnb.fit(X_train, y_train).predict(X_test)
 >>> print("Number of mislabeled points out of a total %d points : %d"
 ... % (X_test.shape[0], (y_test != y_pred).sum()))
 Number of mislabeled points out of a total 75 points : 4

## Multinomial Naive Bayes

implements the naive Bayes algorithm for multinomially distributed data, and is one of the two classic naive Bayes variants used in
text classification (where the data are typically represented as word vector counts, although tf-idf vectors are also known to work well in practice).
The distribution is parametrized by vectors $\theta_y = (\theta_{y1},\ldots,\theta_{yn})$ for each class $y$, where $n$ is the number of features (in text classification, the size of the vocabulary) and $\theta_{yi}$ is the probability $P(x_i \mid y)$ of feature $i$ appearing in a sample belonging to class $y$.

The parameters $\theta_y$ is estimated by a smoothed version of maximum likelihood, i.e. relative frequency counting:

$$ \hat{\theta}_{yi} = \frac{ N_{yi} + \alpha}{N_y + \alpha n} $$

where $N_{yi} = \sum_{x \in T} x_i$ is the number of times feature $i$ appears in a sample of class $y$
in the training set $T$, and $N_{y} = \sum_{i=1}^{n} N_{yi}$ is the total count of all features for class $y$.

The smoothing priors $\alpha \ge 0$ accounts for features not present in the learning samples and prevents zero probabilities
in further computations. Setting $\alpha = 1$ is called Laplace smoothing, while $\alpha < 1$ is called Lidstone smoothing.

## Complement Naive Bayes

$ComplementNB$ implements the complement naive Bayes (CNB) algorithm. CNB is an adaptation of the standard multinomial naive Bayes (MNB) algorithm
that is particularly suited for imbalanced data sets. Specifically, CNB uses statistics from the *complement* of each class to compute the model`s weights.
The inventors of CNB show empirically that the parameter estimates for CNB are more stable than those for MNB. Further, CNB regularly outperforms MNB (often
by a considerable margin) on text classification tasks. The procedure for calculating the weights is as follows:

$$ \hat{\theta}_{ci} = \frac{\alpha_i + \sum_{j:y_j \neq c} d_{ij}}
 {\alpha + \sum_{j:y_j \neq c} \sum_{k} d_{kj}}$$

$$ w_{ci} = \log \hat{\theta}_{ci}$$

$$ w_{ci} = \frac{w_{ci}}{\sum_{j} |w_{cj}|}$$

where the summations are over all documents $j$ not in class $c$, $d_{ij}$ is either the count or tf-idf value of term $i$ in document
$j$, $\alpha_i$ is a smoothing hyperparameter like that found in MNB, and $\alpha = \sum_{i} \alpha_i$. The second normalization addresses
the tendency for longer documents to dominate parameter estimates in MNB. The classification rule is:

$$ \hat{c} = \arg\min_c \sum_{i} t_i w_{ci} $$

i.e., a document is assigned to the class that is the *poorest* complement match.

## References:

 * Rennie, J. D., Shih, L., Teevan, J., & Karger, D. R. (2003).  $Tackling the poor assumptions of naive bayes text classifiers.   <https://people.csail.mit.edu/jrennie/papers/icml03-nb.pdf>$_
 In ICML (Vol. 3, pp. 616-623).


## Bernoulli Naive Bayes

$BernoulliNB$ implements the naive Bayes training and classification algorithms for data that is distributed according to multivariate Bernoulli distributions; i.e., there may be multiple features but each one is assumed to be a binary-valued (Bernoulli, boolean) variable. Therefore, this class requires samples to be represented as binary-valued feature vectors; if handed any other kind of data, a `BernoulliNB` instance
may binarize its input (depending on the `binarize` parameter). 

The decision rule for Bernoulli naive Bayes is based on

$$ P(x_i \mid y) = P(i \mid y) x_i + (1 - P(i \mid y)) (1 - x_i)$$

which differs from multinomial NB`s rule in that it explicitly penalizes the non-occurrence of a feature $i$ that is an indicator for class $y$, where the multinomial variant would simply ignore a non-occurring feature.

In the case of text classification, word occurrence vectors (rather than word count vectors) may be used to train and use this classifier. `BernoulliNB` might perform better on some datasets, especially those with shorter documents.
It is advisable to evaluate both models, if time permits.

## References:

 * C.D. Manning, P. Raghavan and H. Schütze (2008). Introduction to Information Retrieval. Cambridge University Press, pp. 234-265.
 * A. McCallum and K. Nigam (1998). $A comparison of event models for Naive Bayes text classification. <http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.46.1529>$_
 Proc. AAAI/ICML-98 Workshop on Learning for Text Categorization, pp. 41-48.
 * V. Metsis, I. Androutsopoulos and G. Paliouras (2006). $Spam filtering with Naive Bayes -- Which Naive Bayes? <http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.61.5542>$_ 3rd Conf. on Email and Anti-Spam (CEAS).


## Categorical Naive Bayes

$CategoricalNB$ implements the categorical naive Bayes algorithm for categorically distributed data. It assumes that each feature, which is described by the index $i$, has its own categorical distribution. 

For each feature $i$ in the training set $X$,  $CategoricalNB$ estimates a categorical distribution for each feature i of X conditioned on the class y. The index set of the samples is defined as $J = \{ 1, \dots, m \}$, with $m$ as the number of samples.

The probability of category $t$ in feature $i$ given class $c$ is estimated as:

$$ P(x_i = t \mid y = c \: ;\, \alpha) = \frac{ N_{tic} + \alpha}{N_{c} + \alpha n_i},$$

where $N_{tic} = |\{j \in J \mid x_{ij} = t, y_j = c\}|$ is the number of times category $t$ appears in the samples $x_{i}$, which belong to class $c$, $N_{c} = |\{ j \in J\mid y_j = c\}|$ is the number of samples with class c, $\alpha$ is a smoothing parameter and $n_i$ is the number of available categories of feature $i$.

`CategoricalNB` assumes that the sample matrix $X$ is encoded (for instance with the help of  `OrdinalEncoder`) such that all categories for each feature $i$ are represented with numbers $0, ..., n_i - 1$ where $n_i$ is the number of available categories of feature $i$.

# Types of Naive Bayes Classifier:

## Multinomial Naive Bayes:
This is mostly used for document classification problem, i.e whether a document belongs to the category of sports, politics, technology etc. The features/predictors used by the classifier are the frequency of the words present in the document.


## Bernoulli Naive Bayes:
This is similar to the multinomial naive bayes but the predictors are boolean variables. The parameters that we use to predict the class variable take up only values yes or no, for example if a word occurs in the text or not.

## Gaussian Naive Bayes:
When the predictors take up a continuous value and are not discrete, we assume that these values are sampled from a gaussian distribution.

$$P(x_i|y)=\frac{1}{\sqrt{2\pi \sigma^2_y}} e^{\left(\frac{-\left(x_i-\mu_y\right)^2}{2\sigma_y^2}\right)}$$

# import libraies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score

# Importing Data set

In [5]:
dataset=pd.read_csv("data/Social_Network_ads.csv")
X=dataset.iloc[:,2:-1]
y=dataset.iloc[:,-1]
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


# Splitting

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25)
X_train.head()

,Age,EstimatedSalary
108,26,86000
87,28,85000
15,29,80000
10,26,80000
206,55,130000


# Feature Scaling

In [8]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

# Training Naive Baye's Classifier


In [9]:
classifier=GaussianNB()
classifier.fit(X_train,y_train)

GaussianNB()

# Prediction

In [10]:
y_pred=classifier.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], dtype=int64)

# Confusion Matrix and accuracy

In [11]:
cm=confusion_matrix(y_test,y_pred)
cm

array([[59,  4],
       [ 8, 29]], dtype=int64)

In [12]:
accuracy_score(y_test,y_pred)

0.88

# Applications 

Naive Bayes algorithms are mostly used in sentiment analysis, spam filtering, recommendation systems etc. They are fast and easy to implement but their biggest disadvantage is that the requirement of predictors to be independent. In most of the real life cases, the predictors are dependent, this hinders the performance of the classifier.